# Multi Agent Collaboration - Setup
To speed up the lab, we will setup a Bedrock Knowledge Base

### Bedrock Knowledge Base
This will serve as our mock source for information to gather facts about companies, people, products, etc. In a real-world scenario, this would be replaced by API calls to internal and third-party Information provider like LexisNexis, Thomson Reuters Westlaw, Bloomberg Terminal, Factiva, etc.

All the cells in this notebook can take around 10 minutes to execute. Run all cells now and make sure all cells have executed before opening the next notebook.


## Environment Setup

First we ensure that all Python packages required for this notebook are installed, please ignore any errors.

In [ ]:
%pip install -r ../../requirements.txt

### Loading Environment Variables

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()])

## AWS Service Setup

Now we setup our SDK to communicate with various AWS services

In [ ]:
##%%
import boto3
import json
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

## Knowledge Bases for Bedrock Setup
Next we import some of the help functions that have been written to create Bedrock Knowledge Bases


In [ ]:
##%%

sys.path.insert(0, ".")
sys.path.insert(1, "..")
sys.path.insert(2, "../..")

from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)

kb = KnowledgeBasesForAmazonBedrock()

### Knowledge Base Configuration
Let's start filling information about the Knowledge Base we're about to create

In [ ]:
##%%
knowledge_base_name = f'lab7-mac-kb'
knowledge_base_description = "KB containing information about entities like companies, people, and products"
s3_bucket_name = f"labs-bucket-{region}-{account_id}"
bucket_prefix = "data/kb/mac/"

Let's make the Knowledge Base, this command can take a while to finish running.
Behind the scenes, Bedrock is spinning up:
1. OpenSearch Serverless Collection
2. OpenSearch Vector Index
3. Bedrock Knowledge Base

In [ ]:
##%%

kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    s3_bucket_name,
    "amazon.titan-embed-text-v2:0",
    bucket_prefix
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

We have synthetically generated mock information regarding companies, products, and people to simulate the output you may see from an information service. These have been stored in the `information_sources` directory.

We will now upload the mock information to the S3 bucket connected to our knowledge base.

In [ ]:
def upload_directory(path, bucket_name, bucket_prefix):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload, bucket_name, f"{bucket_prefix}{file}")

In [ ]:
upload_directory("../information_sources", s3_bucket_name, bucket_prefix)

It's time to sync that data and ingest it into the vector store.

In [ ]:
kb.synchronize_data(kb_id, ds_id)

Let's set up some variables which will be used later

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']
print(kb_id)
print(kb_info)


## Saving information
Let's store the variables that will be used in other notebooks:

In [ ]:
%store kb_id

## Summary

We have established the foundational infrastructure for a multi-agent collaboration News Writer Agent which transforms raw news facts into 
comprehensive, well-researched articles. The completed tasks include:

1.  Created a Bedrock Knowledge Base to serve as an information repository for entities (companies, people, products)
1.  Populated the knowledge base with synthetic data that simulates real-world information sources
1.  Configured vector embeddings using Amazon Titan for efficient semantic search capabilities
1.  Set up data synchronization between S3 storage and the knowledge base

### 📋 Next Steps

1.  Run the main workflow: Use multi_agent_colloboration.ipynb to execute the multi-agent PR generation
1.  Load stored variables: The main notebook will automatically load kb_id and other stored variables
1.  Monitor performance: Check  quality and optimize the agents as needed
